In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

In [ ]:
!pip install langchain langchainhub langchain-community langchain-core ollama faiss-cpu PyMuPDF requests

In [ ]:
import os
import requests
from langchain.embeddings import OllamaEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOllama
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


pdf_url = "https://arxiv.org/pdf/2306.15595.pdf"
pdf_path = "sample.pdf"

if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)

# Load and split the PDF
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# Create embeddings and FAISS vector store
embedding_func = OllamaEmbeddings(model="phi4")
vector_store = FAISS.from_documents(chunks, embedding_func)
retriever = vector_store.as_retriever()

# Short-term memory
short_term_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# LLM setup
llm = ChatOllama(model="phi4")

# long term memory setup
# long_term_vector_store = FAISS.from_texts([], embedding_func)
# long_term_memory = VectorStoreRetrieverMemory(
#     retriever=long_term_vector_store.as_retriever(),
#     memory_key="history",
#     input_key="question"
# )

# # Step 6: Fact extractor
# fact_prompt = PromptTemplate(
#     input_variables=["question", "answer"],
#     template="""
# You are a memory manager. A user asked a question and got an answer.
# Your task is to extract any important factual info about the user or knowledge
# that should be remembered for future conversations.

# User Question: {question}
# Bot Answer: {answer}

# Only extract facts worth storing (e.g., preferences, facts, interests, tasks).
# If nothing useful, return "None".

# Important Fact to Store:
# """
# )

# fact_extractor = ChatOllama(model="phi4")

# Custom prompt to include conversation history
prompt_template = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
You are a helpful AI assistant. Use the following context and chat history to answer the question. Remember details about the user as well, and answer from them if asked.

Chat History:
{chat_history}

Context:
{context}

Question:
{question}

Answer:
"""
)

# QA Chain with custom prompt and memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=short_term_memory,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    chain_type="stuff"
)

# Chat function
def ask_chat_bot(user_query):
    response = qa_chain.invoke({"question": user_query})
    return response['answer']

# change in the function if I wanna add long term memory
# def ask_chat_bot(user_query):
#     response = qa_chain.run(user_query)

#     # Extract factual info from the conversation
#     fact_input = fact_prompt.format(question=user_query, answer=response)
#     extracted_fact = fact_extractor.invoke(fact_input).content.strip()

#     if extracted_fact and extracted_fact.lower() != "none":
#         long_term_memory.save_context({"question": user_query}, {"output": extracted_fact})

#     return response

# Command line chat interface
if __name__ == "__main__":
    print("🤖 Hello! Ask me anything based on the PDF or general info. Type 'quit' or 'exit' to close the chatbot.")
    while True:
        user_query = input("\n🧑 You: ")
        if user_query.lower() in ["exit", "quit"]:
            print("👋 Bye!")
            break
        bot_reply = ask_chat_bot(user_query)
        print("🤖 Bot:", bot_reply)

<ipython-input-3-39b093c21ff0>:28: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_func = OllamaEmbeddings(model="phi4")
<ipython-input-3-39b093c21ff0>:33: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  short_term_memory = ConversationBufferMemory(
<ipython-input-3-39b093c21ff0>:39: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~la

🤖 Hello! Ask me anything based on the PDF or general info. Type 'quit' or 'exit' to close the chatbot.

🧑 You: my favorite color is blue
🤖 Bot: It's great to know that your favorite color is blue! Blue often represents calmness and serenity, but it can also symbolize depth and stability. Is there anything specific about the color blue you'd like to discuss or explore? If you have any other questions or need assistance with something else, feel free to ask!

If this was in response to a conversation context about preferences or personal details, please let me know if there's more you'd like to share or delve into regarding that.

🧑 You: whats my favorite color?
🤖 Bot: Your favorite color is blue! Blue often represents calmness, serenity, depth, and stability. If there's anything specific you'd like to discuss or explore about this color, feel free to let me know. If you have any other questions or need assistance with something else, I'm here to help!



🧑 You: summarize the paper in 5 